In [1]:
import os
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from datetime import datetime


## Функция для аугментации датасета

In [2]:
# Вспомогательные функции
blist = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90+']

# Функция назначения номера возрастной группы по значению возраста
def binagex(a):
    x = a // 10
    x = 9 if x > 9 else x
    return x

# утилита
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)


In [3]:
'''
Функция для аугментации датасета
параметры:
    ds: начальный pd.resFrame
    delcols: список имен колонок, которые надо удалить из датасета
    необязательные именованные аргументы
        1. имя_колонки=True - Если в функции встроен генератор колонки с таким именем, то он будет выполняться
        2. creators=[список внешних функций, которые будут выполнены]

Возвращает: pd.resFrame

Пример:
        # Удаление трех колонок, добавление групп по возрвстам (OHE)
        data = DS_Gen(data, delcols=['institution_id','diagnos', 'sss'], agebin=True)
        # Без удаления, добавить колонки зодиака и смещения критического возраста
        data = DS_Gen(data, critical_age=True, zodiak=True)
'''
def DS_Gen(ds, delcols=[], **kwargs):
    res = ds.copy()      #чтение исходного датасета

    # Удаление колонок по списку
    #
    _dc = []
    for c in delcols:
        if c in res.columns:
            _dc.append(c)
        else:
          #  raise f'{c} не найдена в исходном датасете'
          print( f'{c} не найдена в исходном датасете')
    res.drop(columns=_dc, axis=1, inplace=True )
    print(f'Удалены: {_dc}')

    # Обработка встроенных колонок
    # ---------------------------------
    # Сейчас реализованы:
    #             critical_age, agebin, #zodiac

    # Константы
    TR_AGE = 40  # принятый порог критического возраста
    OHE = True   # Развертывать ли agebin в OHE колонки

    def _critical_age_(tr_age=TR_AGE):
        if 'age' in res.columns and 'int' in res.age.dtype.name:
            res['critical_age_l']=res['age'].apply(lambda x: tr_age - x if x < tr_age else 0 ).astype('int8')
            res['critical_age_h']=res['age'].apply(lambda x: x - tr_age if x > tr_age else 0  ).astype('int8')

    def _agebin_(ds, ohe=OHE):
        if 'age' in ds.columns and 'int' in ds.age.dtype.name:
            ds['agebix']=ds['age'].apply(lambda x: binagex(x)).astype('int8')
            ds['agebin'] = ds['agebix'].apply(lambda x: blist[x])
            ds = encode_and_bind(ds, 'agebin') if ohe else ds
        return ds


    ##
    # Добавление встроенных колонок, если признак=True в kwargs
    #
    if 'critical_age' in kwargs and kwargs['critical_age']:
        _critical_age_()
        print(f"добавлены 'critical_age_l', 'critical_age_h'")

    if 'agebin' in kwargs and  kwargs['agebin']:
        res = _agebin_(res,True)
        print(f"добавлены 'agebix' и agebin_OHE векторы")

    # завершение работы функции
    return res

In [17]:
# !pip install zodiac-sign

from zodiac_sign import get_zodiac_sign

# знак зодиака на русском
def zsign(d):
    return get_zodiac_sign(d['birthday'], language='ru_RU')

def zodiac_name(row):
    return get_zodiac_sign(row['birthday'], language='ru_RU')


In [18]:
creators = [zsign, zodiac_name]
# names = creators.__names__
for c in creators:
    print(c.__name__)

zsign
zodiac_name


In [20]:
dz = data.copy()
for c in creators:
    print(c.__name__)
    dz[c.__name__] = dz.apply(c, axis=1)
dz.head()

zsign
zodiac_name


,institution,id,gender,age,dose,date_analyse,prev_INR(MHO),atrial_fibrillation,mitral_valve,aortal_valve,...,d8,deviation_from_median,age_percentile,age_quantile,is_elderly,age_density,age_zscore,birthday,zsign,zodiac_name
0,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-11,3.11,1,0,0,...,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11,Скорпион,Скорпион
1,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-24,3.32,1,0,0,...,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11,Скорпион,Скорпион
2,ГБУЗ РБ Толбазинская ЦРБ,140361892057264,1,50,5.00,2022-11-22,2.01,0,1,0,...,0,-17,5.502392,1,0,0.007255,-1.606331,1972-11-22,Стрелец,Стрелец
3,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-11-21,1.42,1,0,0,...,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21,Скорпион,Скорпион
4,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-12-06,2.56,1,0,0,...,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21,Скорпион,Скорпион


## Тесты

In [4]:
# Загрузка датасета со всеми столбцами из текущей папки
data = pd.read_excel("C:\\Users\\Sovtsov\\Documents\\УИИ\\Стажировка\\dataset_wide_birthday.xlsx")


In [5]:
data.columns

Index(['institution', 'id', 'gender', 'age', 'dose', 'date_analyse',
       'prev_INR(MHO)', 'atrial_fibrillation', 'mitral_valve', 'aortal_valve',
       'date_diff', 'INR(MHO)', 'sample', 'pass_dose', 'institution_id',
       'diagnos', 'sss', 'date_from_1990', 'ageix', 'agebin_0-9',
       'agebin_10-19', 'agebin_20-29', 'agebin_30-39', 'agebin_40-49',
       'agebin_50-59', 'agebin_60-69', 'agebin_70-79', 'agebin_80-89',
       'agebin_90+', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8',
       'deviation_from_median', 'age_percentile', 'age_quantile', 'is_elderly',
       'age_density', 'age_zscore', 'birthday'],
      dtype='object')

In [6]:
# тестовый список колонок для удаления
dellst = [ 'agebix', 'agebin_0-9',
       'agebin_10-19', 'agebin_20-29', 'agebin_30-39', 'agebin_40-49',
       'agebin_50-59', 'agebin_60-69', 'agebin_70-79', 'agebin_80-89',
       'agebin_90+','critical_age_l','critical_age_h' ]



In [7]:
# data = DS_Gen(data)
# display(data.head())

Удалены: []


,institution,id,gender,age,dose,date_analyse,prev_INR(MHO),atrial_fibrillation,mitral_valve,aortal_valve,...,d6,d7,d8,deviation_from_median,age_percentile,age_quantile,is_elderly,age_density,age_zscore,birthday
0,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-11,3.11,1,0,0,...,0,0,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11
1,ГБУЗ РБ Толбазинская ЦРБ,140361892057136,0,57,7.50,2022-11-24,3.32,1,0,0,...,0,0,0,-10,13.602187,1,0,0.017956,-0.919637,1965-11-11
2,ГБУЗ РБ Толбазинская ЦРБ,140361892057264,1,50,5.00,2022-11-22,2.01,0,1,0,...,0,1,0,-17,5.502392,1,0,0.007255,-1.606331,1972-11-22
3,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-11-21,1.42,1,0,0,...,0,0,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21
4,ГБУЗ РБ Толбазинская ЦРБ,140361892057392,0,53,6.25,2022-12-06,2.56,1,0,0,...,0,0,0,-14,8.168148,1,0,0.010555,-1.312034,1969-11-21
